In [ ]:
from multiprocessing import Process, Queue

import numpy as np
from scipy.spatial.distance import cdist, pdist, squareform
from tqdm import tqdm_notebook

from tools.data_loading import load_images, load_labels, dummy_code, load_images_resized
from tools.feature_learning import pins_generation
from tools.hog import hog
from tools.kernels import kernel_matrix
from tools.optimization import find_f
from tools.prediction import pred
from tools.process_images import process_images
from tools.quantization import kmeans
from tools.submission import labels_to_csv
from tools.visualization import imshow, dump_as_png, reshape_as_images

# Data loading

In [ ]:
X_train = load_images(type="train")
n_train = X_train.shape[0]
Y_labels_train = load_labels()
Y_train = dummy_code(Y_labels_train)
n_classes = Y_train.shape[1]

In [ ]:
image_list = reshape_as_images(X_train)

# Visual features

In [ ]:
filter_sigma=0.1
filter_shape=5
hog_cell_size=8
disc_grid=16

In [ ]:
n_images = image_list.shape[0]

In [ ]:
hog_list = []
for i in tqdm_notebook(range(n_images), desc="Images"):
    image = image_list[i,:,:,:]
    hog_list.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid,
            color_grad=True))
    
n_features = hog_list[0].size
X_hog = np.array(hog_list).reshape((n_images, n_features)) 


# Data separation

In [ ]:
training_idx = []
test_idx = []
train_frac = 1
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    selected_indices = np.random.permutation(np.sum(Y_labels_train == dig))
    n_class = len(selected_indices)
    class_indices = np.where(Y_labels_train == dig)[0]
    training_idx += list(map(int, class_indices[selected_indices[:int(train_frac*n_class)]]))
    test_idx += list(map(int,class_indices[selected_indices[int(train_frac*n_class):]]))

In [ ]:
X_sample = X_hog[training_idx, :]
n_sample = X_sample.shape[0]
Y_sample = Y_train[training_idx, :]
Y_labels_sample = Y_labels_train[training_idx]

X_test = X_hog[test_idx, :]
n_test = X_test.shape[0]
Y_labels_test = Y_labels_train[test_idx]

## Pre-computing distances

In [ ]:
train_dist = squareform(pdist(X_sample, "euclidean"))
train_test_dist = cdist(X_sample[:500], X_sample, "euclidean")
test_dist = cdist(X_test, X_sample, "euclidean")

## Pre-computing inner products

In [ ]:
train_inner = X_sample.dot(X_sample.T)
train_test_inner = X_sample[:500].dot(X_sample.T)
test_inner = X_test.dot(X_sample.T)

# Training

## Classifier choice

In [ ]:
classifier_type = "fast_svm"
n_iter=60000

## Kernel choice

In [ ]:
kernel_type = "polynomial"
degree = 4
constant = 5
lamb = 1

In [ ]:
kernel_type="rbf"
sigma = 1
lamb = 1e-8

## Kernel computation

In [ ]:
K_sample = kernel_matrix(
    X_sample,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=train_dist,
    inner=train_inner)

## Optimization

In [ ]:
alpha = np.zeros((n_classes, n_sample))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                           prob_type=classifier_type, lamb=lamb, n_iter=n_iter)

# Evaluation

### Training error

In [ ]:
Y_pred_train = pred(
    X_sample, X_sample[:500], alpha,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=train_test_dist,
    inner=train_test_inner)


Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
prec = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
print("The precision on the train set is of {}".format(prec))

### Testing error

In [ ]:
Y_pred = pred(
    X_sample, X_test, alpha,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=test_dist,
    inner=test_inner)


Y_labels_pred = np.argmax(Y_pred, axis=1)
prec = np.mean(Y_labels_pred == Y_labels_test)
print("The precision on the test set is of {}".format(prec))

## Grid search from hell

In [ ]:
def coordinate_ascent(out_q, dig, **kwargs):
    out_q.put(
        [
            dig,
            find_f(**kwargs)            
        ]
    )

### For polynomial

In [ ]:
kernel_type = "polynomial"
degrees = [4]
n_degrees = len(degrees)
constants = [1]
n_constants = len(constants)
lambs = [1]
n_lambs = len(lambs)
weights = [5, 10, 50, 1000]
n_weights = len(weights)
train_prec = np.zeros((n_degrees, n_constants, n_lambs, n_weights))
test_prec = np.zeros((n_degrees, n_constants, n_lambs, n_weights))
for d in tqdm_notebook(range(n_degrees), desc="Degrees"):
    for c in tqdm_notebook(range(n_constants), desc="Constants"):
        # Kernel matrix computation
        K_sample = kernel_matrix(
            X_sample, kernel_type=kernel_type, degree=degrees[d], constant=constants[c], inner=train_inner)
        for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
            for w in tqdm_notebook(range(n_weights), desc="Weights"):
                # Coordinate ascent with multiprocessing
                alpha = np.zeros((n_classes, n_sample))
                out_q = Queue()
                procs = list()
                for dig in range(n_classes):
                    weights = np.ones(n_sample)
                    weights[Y_labels_sample == dig] *= weights[w]
                    p = Process(
                        target=coordinate_ascent,
                        args=(out_q, dig),
                        kwargs={
                            "K":K_sample,
                            "Y":Y_sample[:, dig],
                            "weights": weights,
                            "prob_type":classifier_type, "lamb":lambs[l], "n_iter":100000
                        }
                    )
                    procs.append(p)
                    p.start()
                for i in range(n_classes):
                    res = out_q.get()
                    alpha[res[0], :] = res[1]

                for p in procs:
                    p.join()
                # Precision on train set
                Y_pred_train = pred(
                    X_sample, X_sample[:500], alpha,
                    kernel_type=kernel_type,
                    degree=degrees[d],
                    constant=constants[c],
                    inner=train_test_inner)

                Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
                train_prec[d, c, l, w] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
                # Precision on test set
                Y_pred= pred(
                    X_sample, X_test, alpha,
                    kernel_type=kernel_type,
                    degree=degrees[d],
                    constant=constants[c],
                    inner=test_inner)

                Y_labels_pred = np.argmax(Y_pred, axis=1)
                test_prec[d, c, l, w] = np.mean(Y_labels_pred == Y_labels_test)

In [ ]:
test_prec

In [ ]:
train_prec

### For RBF

In [ ]:
kernel_type="rbf"
sigmas = [1.5]
n_sigmas = len(sigmas)
weights = [10]
n_weights = len(weights)
lambs = [1e-8]
n_lambs = len(lambs)
train_prec_rbf = np.zeros((n_sigmas, n_lambs, n_weights))
test_prec_rbf = np.zeros((n_sigmas, n_lambs, n_weights))
for s in tqdm_notebook(range(n_sigmas), desc="Sigmas"):
    # Kernel matrix computation
    K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, sigma=sigmas[s], dist=train_dist)
    for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
        for w in tqdm_notebook(range(n_weights), desc="Weights"):
            # Coordinate ascent with multiprocessing
            alpha = np.zeros((n_classes, n_sample))
            out_q = Queue()
            procs = list()
            for dig in range(n_classes):
                weights = np.ones(n_sample)
                weights[Y_labels_sample == dig] *= weights[w]
                p = Process(
                    target=coordinate_ascent,
                    args=(out_q, dig),
                    kwargs={
                        "K":K_sample,
                        "Y":Y_sample[:, dig],
                        "weights": weights,
                        "prob_type":classifier_type, "lamb":lambs[l], "n_iter":100000
                    }
                )
                procs.append(p)
                p.start()
            for i in range(n_classes):
                res = out_q.get()
                alpha[res[0], :] = res[1]

            for p in procs:
                p.join()
            # Precision on train set
            Y_pred_train= pred(X_sample, X_sample[:500], alpha,
                                  kernel_type=kernel_type, sigma=sigmas[s], dist=train_test_dist)
            Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
            train_prec_rbf[s, l, w] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
            # Precision on test set
            Y_pred = pred(X_sample, X_test, alpha,
                                  kernel_type=kernel_type, sigma=sigmas[s], dist=test_dist)


            Y_labels_pred = np.argmax(Y_pred, axis=1)
            test_prec_rbf[s, l, w] = np.mean(Y_labels_pred == Y_labels_test)

In [ ]:
test_prec_rbf

In [ ]:
train_prec_rbf

# Prediction

In [ ]:
X_eval = load_images(type="test")
n_eval = X_eval.shape[0]

image_list_eval = reshape_as_images(X_eval)

# Visual features for submission
hog_list_eval = []
for i in tqdm_notebook(range(n_eval), desc="Submission images"):
    image = image_list_eval[i,:,:,:]
    hog_list_eval.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid,
            color_grad=True))
    
n_features = hog_list_eval[0].size
X_hog_eval = np.array(hog_list_eval).reshape((n_eval, n_features)) 

In [ ]:
Y_eval = pred(X_sample, X_hog_eval, alpha,
                      kernel_type=kernel_type, degree=degree, constant=constant, sigma=sigma)


Y_labels_eval = np.argmax(Y_eval, axis=1)


# Submission
labels_to_csv(Y_labels_eval, kernel=kernel_type, algo="svm_hog",user="Zac_color_grad")